In [1]:
import os
import sys
import tqdm
import pandas as pd

In [2]:
REBADD_LIB_PATH = os.path.abspath(os.pardir)
if REBADD_LIB_PATH not in sys.path:
    sys.path = [REBADD_LIB_PATH] + sys.path

from rebadd.evaluate import evaluate_sr_nov_div

In [3]:
## Test data (navitoclax, ABT-737)
filepath_ref = os.path.join(os.pardir, 'data', 'zinc15', 'zinc15_test.txt')
df_ref = pd.read_csv(filepath_ref, header=None)

referece_smiles_iter = df_ref.iloc[:,0].values.tolist()

print(len(referece_smiles_iter))

2


In [4]:
input_dir = 'outputs_7_calculate_properties_generated'

modelnames = [
    'frag+reinforce+scst+offpolicy',
]

filenames = [f'smi_after.csv.{num}' for num in range(10)]

In [5]:
frames = []

for modelname in modelnames:
    
    for filename in filenames:
        
        filepath = os.path.join(input_dir, modelname, filename)
        
        df = pd.read_csv(filepath)
        
        df = df.loc[:,('smiles', 'bcl2', 'bclxl', 'bclw')]
        df.loc[:,'model'] = modelname
        df.loc[:,'checkpoint'] = filename.split('.')[-1]
        
        frames.append(df)

In [6]:
data = []

for df in tqdm.tqdm(frames):

    s_sr, s_nov, s_div = evaluate_sr_nov_div(df, referece_smiles_iter, 'bcl2_bclxl_bclw')
    
    data.append({'SR':s_sr, 'Nov':s_nov, 'Div':s_div, 'Model':df.loc[0,'model'], 'Ckpt':df.loc[0,'checkpoint']})

100%|██████████| 10/10 [00:07<00:00,  1.32it/s]


In [7]:
df_records = pd.DataFrame(data)
df_records.loc[:,'HMean'] = (df_records.loc[:,'SR'] * df_records.loc[:,'Nov'] * df_records.loc[:,'Div']) ** 0.333

df_records

,SR,Nov,Div,Model,Ckpt,HMean
0,0.9866,1.0,0.517870,frag+reinforce+scst+offpolicy,0,0.799622
1,0.9854,1.0,0.524482,frag+reinforce+scst+offpolicy,1,0.802681
2,0.9870,1.0,0.502878,frag+reinforce+scst+offpolicy,2,0.791945
3,0.9860,1.0,0.503612,frag+reinforce+scst+offpolicy,3,0.792062
4,0.9860,1.0,0.519443,frag+reinforce+scst+offpolicy,4,0.800268
5,0.9838,1.0,0.526200,frag+reinforce+scst+offpolicy,5,0.803121
6,0.9862,1.0,0.510418,frag+reinforce+scst+offpolicy,6,0.795664
7,0.9862,1.0,0.518040,frag+reinforce+scst+offpolicy,7,0.799601
8,0.9876,1.0,0.515100,frag+reinforce+scst+offpolicy,8,0.798464
9,0.9880,1.0,0.514526,frag+reinforce+scst+offpolicy,9,0.798276


In [8]:
df_records.groupby('Model').mean(numeric_only=True)

,SR,Nov,Div,HMean
Model,,,,
frag+reinforce+scst+offpolicy,0.98628,1.0,0.515257,0.79817


In [9]:
df_records.groupby('Model').std(numeric_only=True)

,SR,Nov,Div,HMean
Model,,,,
frag+reinforce+scst+offpolicy,0.001175,0.0,0.007816,0.003885
